# Head Velocity
Notebook for clean attempt of checking the distribution of head velocit and stimulus related behavior of the animal.

In [2]:
# data loading
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats, integrate

%matplotlib inline
sns.set(color_codes=True)

path = '../datasets/'
dfrat     = pd.read_hdf(path+'relationalDatabase.h5', 'Rat_Behavior').set_index('index')
dfevent   = pd.read_hdf(path+'relationalDatabase.h5', 'Events').set_index('index')
dfsession = pd.read_hdf(path+'relationalDatabase.h5', 'Sessions').set_index('index')

In [3]:
# in use functions
def angle_cos(V1, V2):
    CosTheta = np.dot(V1,V2) / (np.linalg.norm(V1) * np.linalg.norm(V2))
    return CosTheta


def angle_zero(df):
    V1 = np.array([df.lx1, 0, df.lz1])
    V2 = np.array([1     , 0, 0     ])
    df['theta0'] = np.arccos(angle_cos(V1,V2))
    return df


def decompose_vec_df(df):
    # get two vectors aligned with Y and Z axes
    Vo = np.array([df.X_Ori, df.Y_Ori, df.Z_Ori])  
    Vz = np.array([df.X_Ori, df.Y_Ori, 0       ])  
    Vx = np.array([0       , df.Y_Ori, df.Z_Ori])

    # find the angles between them and orientation vector 
    df['lx1'] = angle_cos(Vo, Vz)
    df['lz1'] = angle_cos(Vo, Vx)
    return df


## Angular velocity calculations
Calculating vectors, angle, velocity for comparison.

In [4]:
pd.options.mode.chained_assignment = None 

## VECTORS WITHOUT Y
dfOri = pd.DataFrame()
dfV = dfrat.copy()

for name, dd in dfV.groupby('session_id'):
    print(name)
    
    dd['dT'] = dd['Time'].diff(-1)    
    
    # calculating lenghts of VecX and VecZ and creation of shifted vectors  #dd = dd.apply(decompose_vec_df, axis=1)
    dd['zero'] = dd.X_Ori * 0
    Vo = np.array([dd.X_Ori, dd.Y_Ori, dd.Z_Ori])  
    Vz = np.array([dd.X_Ori, dd.Y_Ori, dd.zero ])  
    Vx = np.array([dd.zero , dd.Y_Ori, dd.Z_Ori])
    
    dd['lVx'] = np.linalg.norm(Vo) * np.linalg.norm(Vx)
    dd['lVz'] = np.linalg.norm(Vo) * np.linalg.norm(Vz)
    
    dd['dx1'] =        0*dd.X_Ori + dd.Y_Ori*dd.Y_Ori + dd.Z_Ori*dd.Z_Ori
    dd['dz1'] = dd.X_Ori*dd.X_Ori + dd.Y_Ori*dd.Y_Ori +        0*dd.Z_Ori
    dd['lx1'] = dd['dx1'] / dd['lVx']
    dd['lz1'] = dd['dz1'] / dd['lVz']
        
    # propagate the sign of a X, Z to new vector
    dd['lx1'] = dd['lx1'] * (dd['X_Ori'] / np.absolute(dd['X_Ori']))
    dd['lz1'] = dd['lz1'] * (dd['Z_Ori'] / np.absolute(dd['Z_Ori']))

    # normalize the data to unit vector form
    dd['V1'] = np.linalg.norm(np.array([dd.lx1, 0, dd.lz1]))
    dd['lx1'] = dd.lx1 / dd.V1
    dd['lz1'] = dd.lz1 / dd.V1

    dfOri = pd.concat([dfOri, dd], axis=0, ignore_index=True)

dfOri = dfOri.replace([np.inf, -np.inf], np.nan).dropna()


0
1
2
3
4
5
6
7


In [5]:
## ANGLES and VELOCITY
dfOri2 = pd.DataFrame()

# calculatinig the angles between X axis and the vectors
for name, dd in dfOri.groupby('session_id'):
    print(name)
    
#     dd = dd.apply(angle_zero, axis=1) # calculation with origin
    V1 = np.array([dd.lx1, 0, dd.lz1])
    V2 = np.array([1     , 0, 0     ])
    
    dd['theta0'] = np.arccos(dd.lx1 / (np.linalg.norm(V1) * np.linalg.norm(V2)))
    
    dd['theta1'] = dd.theta0.shift(1)
    dd['theta']  = dd.theta0 - dd.theta1
    dd.drop({'theta0','theta1'}, axis=1, inplace=True)
    
    # angular velocity between two vectors
    dd['U'] = np.degrees(dd.theta)/ dd.dT
    
    dfOri2 = pd.concat([dfOri2, dd], axis=0, ignore_index=True)
    
dfOri2 = dfOri2.replace([np.inf, -np.inf], np.nan).dropna()


0
1
2
3
4
5
6
7


## Stimulus-related behavior
Looking for frames detected as SRB.

SRB = {f E FRAMES: ||Vhead - Vstim|| < Dmax}

In [1]:
# ## COMPARISON WITH EVENT DATA
# dfOri3 = pd.DataFrame()
# DmaxPerc = 0.2

# dfeventsClean = dfevent.drop({'MotiveExpTimeSecs', 'session_id','Time'}, axis=1)
# dfV = pd.merge(dfOri2, dfeventsClean, on='Frame')

# dfV.head()


NameError: name 'pd' is not defined

In [ ]:
# # calculatinig the angles between X axis and the vectors
# for name, dd in dfV.groupby('session_id'):
#     # substraction
#     dd['subU'] = dd.speed - dd.U
    
#     # setting Dmax as 20% of velocity at given moment // what about 0
#     dd['Dmax'] = dd.speed * DmaxPerc
#     dd['Dmax'] = dd.apply(lambda row: 0.1 if row.Dmax == 0 else row.Dmax, axis=1)
#     dd['SRB']  = np.absolute(dd.sub) < np.absolute(dd.Dmax)

#     dfOri3 = pd.concat([dfOri3, dd], axis=0, ignore_index=True)
    
# dfOri3 = dfOri3.replace([np.inf, -np.inf], np.nan).dropna()